In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers


In [ ]:
train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")


In [ ]:
train.head()


In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())


In [ ]:
x_train = train["comment_text"]

y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

x_test = test["comment_text"]


In [ ]:
max_feature = 20000

tokenizer = Tokenizer(num_words = max_feature)
tokenizer.fit_on_texts(list(x_train))
tokenized_train = tokenizer.texts_to_sequences(x_train)
tokenized_test = tokenizer.texts_to_sequences(x_test)


In [ ]:
tokenized_train[:1]


In [ ]:
maxlen = 50
x_train = pad_sequences(tokenized_train, maxlen = maxlen)
x_test = pad_sequences(tokenized_test, maxlen = maxlen)


In [ ]:
input = Input(shape = (maxlen,))


In [ ]:
embed_size = 128
x = Embedding(max_feature, embed_size)(input)


In [ ]:
x = LSTM(60, return_sequences = True, name = "lstm_layer")(x)


In [ ]:
x = GlobalMaxPool1D()(x)


In [ ]:
x = Dropout(0.1)(x)


In [ ]:
x = Dense(50, activation = "relu")(x)


In [ ]:
#Once again we have implemented dropout to prevent overfitting

x = Dropout(0.1)(x)


In [ ]:
x = Dense(6, activation = "sigmoid")(x)


In [ ]:
model = Model(inputs = input, outputs = x)
model.compile(loss = "binary_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])


In [ ]:
batch_size = 32
epochs = 2
model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, validation_split = 0.1)


In [ ]:
y_pred = model.predict(x_test,batch_size=32)


In [ ]:
submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('submission.csv', index=False)
